# TDC 105: Oracle 

[Kexin](https://twitter.com/KexinHuang5)

Welcome to the TDC community! In this tutorial, we will cover the basics of TDC data functions and after this tutorial, you will be able to leverage most of the useful functions supported!

We assume you have familiarize yourself with the installations and data loaders. If not, please visit [TDC 101 Data Loaders](https://github.com/mims-harvard/TDC/blob/master/tutorials/TDC_101_Data_Loader.ipynb) first!

First, we introduce oracle. xxxxxxx. We mainly implement in the following three kinds of oracles:

* `Molecular Property Oracle` 

* `Distribution Learning Oracle` 

* `xxx` 


## Import the oracle

In [2]:
from tdc.oracle import Oracle


## Molecular Property Oracle

It includes:
    
* `QED`

* `LogP`


In [4]:
oracle = Oracle(name = 'gsk3')


Found local copy...


## Distribution Learning Oracle


It includes:
    
* `Novelty`

* `Validity`


In [6]:
smiles_lst = ['CC(C)(C)[C@H]1CCc2c(sc(NC(=O)COc3ccc(Cl)cc3)c2C(N)=O)C1', \
			  'C[C@@H]1CCc2c(sc(NC(=O)c3ccco3)c2C(N)=O)C1', \
			  'CCNC(=O)c1ccc(NC(=O)N2CC[C@H](C)[C@H](O)C2)c(C)c1', \
			  'C[C@@H]1CCN(C(=O)CCCc2ccccc2)C[C@@H]1O']


oracle = Oracle(name = 'validity')
print(oracle(smiles_lst))

1.0
